In [2]:
from bs4 import BeautifulSoup as bs
from lxml import etree
from pprint import pprint
from pyquery import PyQuery as pq
from selenium import webdriver
from urllib.parse import urlencode
import json
import os
import pandas as pd
import re
import requests
import time

In [3]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

In [4]:
region_map = {
    '台北市':'1',
    '基隆市':'2',
    '新北市':'3',
    '新竹市':'4',
    '新竹縣':'5',
    '桃園市':'6',
    '苗栗縣':'7',
    '台中市':'8',
    '彰化縣':'10',
    '南投縣':'11',
    '嘉義市':'12',
    '嘉義縣':'13',
    '雲林縣':'14',
    '台南市':'15',
    '高雄市':'17',
    '屏東縣':'19',
    '宜蘭縣':'21',
    '台東縣':'22',
    '花蓮縣':'23',
    '澎湖縣':'24',
    '金門縣':'25',
    '連江縣':'26',
}

In [5]:
client = requests.session()

In [6]:
def get_var(client,region,headers):
    index_url='https://rent.591.com.tw/?kind=0&regionid='+region_map[region]
    res = client.get(index_url,headers=headers)
    res.encoding = 'utf-8'
    res.cookies['urlJumpIp']= region_map[region]
#     print(res.cookies.get_dict())
    soup = bs(res.text, "lxml")
#     print(soup.select('head > meta'))

    totalRows = soup.select('#container > section.listBox > div > div.listLeft > div.page-limit > div > a.pageNext')[0]['data-total']
    CSRF_TOKEN = [i['content'] for i in soup.select('head > meta[name]') if i['name']=="csrf-token"][0]
    return (totalRows ,CSRF_TOKEN)

In [61]:
def get_searchUrl(firstRow,region,headers):
    params = {
        'is_new_list': 1,
        'type': 1,
        'kind':0,
        'searchtype':1,
        'region':region_map[region],
        'firstRow':(firstRow*30),
        'totalRows':get_var(client,region,headers)[0]
    }
    base_url = 'https://rent.591.com.tw/home/search/rsList?'
    url = base_url + urlencode(params)
    return url

In [62]:
get_searchUrl(0,'新北市',headers)

'https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=0&totalRows=9371'

In [63]:
from pprint import pprint

In [64]:
import math

In [76]:
def get_searchUrlist(client,region,headers):
    vars = get_var(client,region,headers)
    headers['X-CSRF-Token'] = vars[1]
    times = math.floor(int(vars[0])/30) +1 if int(vars[0])%30 !=0 else math.floor(int(vars[0])/30)
    searchUrlist=[]
    for firstRow in range(times):
        searchUrl = get_searchUrl(firstRow,region,headers)
        print(searchUrl)
        res_search = client.get(searchUrl,headers=headers)
        res_search.encoding = 'utf-8'
        data = json.loads(res_search.text)
#         print(data['data']['data'][0]['post_id'])
        rent_list = [(i['post_id'],'https://rent.591.com.tw/rent-detail-'+str(i['post_id'])+'.html') for i in data['data']['data']]
#         print(rent_list[0])
        searchUrlist.extend(rent_list)     
    return searchUrlist
        

In [79]:
%time searchUrl = get_searchUrlist(client,'新北市',headers)

https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=0&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=30&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=60&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=90&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=120&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=150&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=180&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=210&totalRows=9372
https://rent.591.com.tw/home/search/r

https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=2040&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=2070&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=2100&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=2130&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=2160&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=2190&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=2220&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=2250&totalRows=9372
https://rent.591.com.tw/

https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=4080&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=4110&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=4140&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=4170&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=4200&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=4230&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=4260&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=4290&totalRows=9372
https://rent.591.com.tw/

https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=6120&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=6150&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=6180&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=6210&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=6240&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=6270&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=6300&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=6330&totalRows=9372
https://rent.591.com.tw/

https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=8160&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=8190&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=8220&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=8250&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=8280&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=8310&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=8340&totalRows=9372
https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=8370&totalRows=9372
https://rent.591.com.tw/

In [82]:
searchUrl[0]

(8909835, 'https://rent.591.com.tw/rent-detail-8909835.html')

In [80]:
len(searchUrl)

9371